# Compound  Counts

Count distinct parent compounds

In [1]:
from local_utils.file_utils import backup_file

### Configuration

In [2]:
# ChEMBL connection...

engine = create_engine(open('database.txt').read().strip())

### Reload targets

In [3]:
targets = pd.read_pickle('chembl_targets.pkl')

targets.shape

(377, 9)

In [4]:
symbols = pd.read_pickle('unique_symbols.pkl')

symbols.shape

(215, 3)

## Add counts by symbol

_N.B._ Counts are of distinct parent compunds

In [5]:
# Counts are done in RDBMS...

sql = open('SQL/tt_count.sql').read().strip()

print(sql)

----------------------------------------------------------------------------------------------------
-- Count by ChEMBL Target ID
----------------------------------------------------------------------------------------------------

-- Actives

create table tt_count_active
as
select
    a.target_chemblid
  , count(distinct a.parent_cmpd_chemblid) as n_active
from
  tt_chembl_targets x
  join chembl_20_app.chembldb_bioactivity_summary a on x.chembl_id = a.target_chemblid
where
    a.pchembl_value >= 5
group by
  target_chemblid
;

-- Total

create table tt_count_total
as
select
    a.target_chemblid
  , count(distinct a.parent_cmpd_chemblid) as n_total
from
  tt_chembl_targets x
  join chembl_20_app.chembldb_bioactivity_summary a on x.chembl_id = a.target_chemblid
group by
  target_chemblid
;

----------------------------------------------------------------------------------------------------
-- Count by symbol & species
-------------------------------------------------------------------

In [6]:
# Generate counts tables...

### for sql in [y for y in [x.strip() for x in re.sub('^\s*--.*$', '', sql, flags=re.MULTILINE).split(';')] if y]: engine.execute(sql)

### Count by individual ChEMBL target

In [7]:
count_active = pd.read_sql_table('tt_count_active', engine).set_index('target_chemblid')
count_total  = pd.read_sql_table('tt_count_total',  engine).set_index('target_chemblid')

targets = targets.merge(count_active, how='left', left_on='chembl_id', right_index=True).merge(count_total, how='left', left_on='chembl_id', right_index=True).fillna(0)

In [8]:
targets.head()

,symbol,approved_name,targets,n_target,chembl_id,target_type,pref_name,species,exclude,n_active,n_total
0,ABCB1,"ATP-binding cassette, sub-family B (MDR/TAP), member 1","MDR1;7|MDR1;8|ATP-binding cassette, sub-family B (MDR/TAP), member 1;11",1,CHEMBL4302,SINGLE PROTEIN,P-glycoprotein 1,Human,0,879,2841
1,ABCB1,"ATP-binding cassette, sub-family B (MDR/TAP), member 1","MDR1;7|MDR1;8|ATP-binding cassette, sub-family B (MDR/TAP), member 1;11",1,CHEMBL1075229,SINGLE PROTEIN,Multidrug resistance protein 1,Rat,0,0,68
2,ABCB11,"ATP-binding cassette, sub-family B (MDR/TAP), member 11",BSEP;6|BSEP;7|BSEP;8,1,CHEMBL6020,SINGLE PROTEIN,Bile salt export pump,Human,0,3,49
3,ABCB11,"ATP-binding cassette, sub-family B (MDR/TAP), member 11",BSEP;6|BSEP;7|BSEP;8,1,CHEMBL2073674,SINGLE PROTEIN,Bile salt export pump,Rat,0,9,27
4,ABCB4,"ATP-binding cassette, sub-family B (MDR/TAP), member 4",MDR3;7|MDR3;8,1,CHEMBL1743129,SINGLE PROTEIN,Multidrug resistance protein 3,Human,0,0,6


In [9]:
# Count targets having thirty or more distint active parent compounds...

[len(set(targets.query("species == @x and n_active >= 30")['symbol'].values)) for x in ['Human', 'Rat']]

[147, 58]

In [10]:
# N.B. No targets are lost by just considering Human targets...

len(set(targets.query("n_active >= 30")['symbol'].values))

147

In [11]:
# Get symbols (Human only)...

found0 = set(targets.query("species == 'Human' and n_active >= 30")['symbol'].values)

###  Count by symbol and species

_I.e._ merge ChEMBL targets for a given symbol/species

In [12]:
count_active_by_symbol_1 = pd.read_sql_table('tt_count_active_by_symbol_1', engine).sort(['symbol', 'species']).reset_index(drop=True)

count_active_by_symbol_1.shape

(274, 3)

In [13]:
count_active_by_symbol_1.head()

,symbol,species,n_active
0,ABCB1,Human,879
1,ABCB11,Human,3
2,ABCB11,Rat,9
3,ABCC2,Human,8
4,ABCC2,Rat,21


In [14]:
# Count symbols (for species) having thirty or more distint active parent compounds...

[len(set(count_active_by_symbol_1.query("species == @x and n_active >= 30")['symbol'].values)) for x in ['Human', 'Rat']]

[148, 58]

In [15]:
# Get symbols (Human only)...

found1 = set(count_active_by_symbol_1.query("species == 'Human' and n_active >= 30")['symbol'].values)

In [16]:
# Symbols 'recovered' by merging targets (Human only)...

recovered1 = set(found1).difference(found0)

len(recovered1)

1

In [17]:
count_active_by_symbol_1.query("symbol in @recovered1 and species == 'Human'")

,symbol,species,n_active
224,PRKAA2,Human,31


In [18]:
targets.query("symbol in @recovered1 and species == 'Human'")

,symbol,approved_name,targets,n_target,chembl_id,target_type,pref_name,species,exclude,n_active,n_total
307,PRKAA2,"protein kinase, AMP-activated, alpha 2 catalytic subunit",AMPK;5,1,CHEMBL3038454,PROTEIN COMPLEX,AMPK alpha1/alpha2,Human,0,0,12
308,PRKAA2,"protein kinase, AMP-activated, alpha 2 catalytic subunit",AMPK;5,2,CHEMBL3038455,PROTEIN COMPLEX,AMPK alpha2/beta1/gamma1,Human,0,14,17
309,PRKAA2,"protein kinase, AMP-activated, alpha 2 catalytic subunit",AMPK;5,3,CHEMBL3038456,PROTEIN COMPLEX,AMPK alpha2/beta2/gamma1,Human,0,0,4
310,PRKAA2,"protein kinase, AMP-activated, alpha 2 catalytic subunit",AMPK;5,4,CHEMBL3038457,PROTEIN COMPLEX,AMPK alpha2/beta2/gamma3,Human,0,0,2
311,PRKAA2,"protein kinase, AMP-activated, alpha 2 catalytic subunit",AMPK;5,5,CHEMBL2116,SINGLE PROTEIN,"AMP-activated protein kinase, alpha-2 subunit",Human,0,17,501


### Count by symbol only

_I.e._ merge ChEMBL targets, across species, for a given symbol

In [19]:
count_active_by_symbol_2 = pd.read_sql_table('tt_count_active_by_symbol_2', engine).sort(['symbol']).reset_index(drop=True)

count_active_by_symbol_2.shape

(182, 2)

In [20]:
count_active_by_symbol_2.head()

,symbol,n_active
0,ABCB1,879
1,ABCB11,11
2,ABCC2,24
3,ABCC3,2
4,ABCC4,8


In [21]:
# Get and count symbols having thirty or more distint active parent compounds...

found2 = set(count_active_by_symbol_2.query("n_active >= 30")['symbol'].values)

len(found2)

151

In [22]:
# Extra symbols 'recovered' by merging targets *and* species...

recovered2 =  set(found2).difference(found0).difference(found1)

len(recovered2)

3

In [23]:
count_active_by_symbol_2.query("symbol in @recovered2")

,symbol,n_active
162,SLC15A2,30
163,SLC22A1,31
165,SLC22A6,35


In [24]:
targets.query("symbol in @recovered2")

,symbol,approved_name,targets,n_target,chembl_id,target_type,pref_name,species,exclude,n_active,n_total
334,SLC15A2,"solute carrier family 15 (oligopeptide transporter), member 2","solute carrier family 15 (H+/peptide transporter), member 2;11",1,CHEMBL1743125,SINGLE PROTEIN,Solute carrier family 15 member 2,Human,0,22,88
335,SLC15A2,"solute carrier family 15 (oligopeptide transporter), member 2","solute carrier family 15 (H+/peptide transporter), member 2;11",1,CHEMBL3325,SINGLE PROTEIN,"Oligopeptide transporter, kidney isoform",Rat,0,10,86
336,SLC22A1,"solute carrier family 22 (organic cation transporter), member 1","OCTs;7|OCT1;8|solute carrier family 22 (organic cation transporter), member 1;11",1,CHEMBL5685,SINGLE PROTEIN,Solute carrier family 22 member 1,Human,0,19,257
337,SLC22A1,"solute carrier family 22 (organic cation transporter), member 1","OCTs;7|OCT1;8|solute carrier family 22 (organic cation transporter), member 1;11",1,CHEMBL2073670,SINGLE PROTEIN,Solute carrier family 22 member 1,Rat,0,16,68
340,SLC22A6,"solute carrier family 22 (organic anion transporter), member 6","solute carrier family 22 (organic anion transporter), member 6;11",1,CHEMBL1641347,SINGLE PROTEIN,Solute carrier family 22 member 6,Human,0,26,92
341,SLC22A6,"solute carrier family 22 (organic anion transporter), member 6","solute carrier family 22 (organic anion transporter), member 6;11",1,CHEMBL1777665,SINGLE PROTEIN,Solute carrier family 22 member 6,Rat,0,13,124


## What targets have been lost?

In [25]:
# Consider only Human targets passing distinct active compound threshold...

lost = set(symbols['symbol']).difference(found1)

len(lost)

67

_I.e._ 215 - 148 = 67

In [26]:
symbols.query("symbol in @lost")

,symbol,approved_name,targets
symbol,,,
ABCB11,ABCB11,"ATP-binding cassette, sub-family B (MDR/TAP), member 11","((BSEP, 6), (BSEP, 7), (BSEP, 8))"
ABCB4,ABCB4,"ATP-binding cassette, sub-family B (MDR/TAP), member 4","((MDR3, 7), (MDR3, 8))"
ABCC2,ABCC2,"ATP-binding cassette, sub-family C (CFTR/MRP), member 2","((MRP2, 7), (MRP2, 8), (ATP-binding cassette, sub-family C (CFTR/MRP), member 2, 11))"
ABCC3,ABCC3,"ATP-binding cassette, sub-family C (CFTR/MRP), member 3","((MRP3, 7), (MRP3, 8))"
ABCC4,ABCC4,"ATP-binding cassette, sub-family C (CFTR/MRP), member 4","((MRP4, 7), (MRP4, 8))"
ABCC6,ABCC6,"ATP-binding cassette, sub-family C (CFTR/MRP), member 6","((MRP6, 8),)"
ABCG5,ABCG5,"ATP-binding cassette, sub-family G (WHITE), member 5","((ABCG5, 7),)"
ABCG8,ABCG8,"ATP-binding cassette, sub-family G (WHITE), member 8","((ABCG8, 7),)"
ATP1A1,ATP1A1,"ATPase, Na+/K+ transporting, alpha 1 polypeptide","((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))"


#### Example: CALCR

In [27]:
targets.query("symbol == 'CALCR'")

,symbol,approved_name,targets,n_target,chembl_id,target_type,pref_name,species,exclude,n_active,n_total
81,CALCR,calcitonin receptor,CGRP1;2.2,1,CHEMBL2111189,PROTEIN COMPLEX,"Amylin receptor AMY1, CALCR/RAMP1",Human,0,1,1
82,CALCR,calcitonin receptor,CGRP1;2.2,2,CHEMBL2364173,PROTEIN COMPLEX,Amylin receptor AMY2; CALCR/RAMP2,Human,0,0,0
83,CALCR,calcitonin receptor,CGRP1;2.2,3,CHEMBL2111190,PROTEIN COMPLEX,Amylin receptor AMY3; CALCR/RAMP3,Human,0,1,1
84,CALCR,calcitonin receptor,CGRP1;2.2,4,CHEMBL1832,SINGLE PROTEIN,Calcitonin receptor,Human,0,14,890
85,CALCR,calcitonin receptor,CGRP1;2.2,1,CHEMBL2107837,PROTEIN COMPLEX,"Amylin receptor AMY1, CALCR/RAMP1",Rat,0,1,1
86,CALCR,calcitonin receptor,CGRP1;2.2,2,CHEMBL2109231,PROTEIN COMPLEX,Amylin receptor AMY2; CALCR/RAMP2,Rat,0,1,1
87,CALCR,calcitonin receptor,CGRP1;2.2,3,CHEMBL2109235,PROTEIN COMPLEX,Amylin receptor AMY3; CALCR/RAMP3,Rat,0,1,1
88,CALCR,calcitonin receptor,CGRP1;2.2,4,CHEMBL2204,SINGLE PROTEIN,Calcitonin receptor,Rat,0,11,14


In [28]:
count_active_by_symbol_1.query("symbol == 'CALCR'")

,symbol,species,n_active
71,CALCR,Human,14
72,CALCR,Rat,12


In [29]:
count_active_by_symbol_2.query("symbol == 'CALCR'")

,symbol,n_active
43,CALCR,16


### In terms of original 'targets' and the tables they appear in

The 'lost' targets mainly seem to be...

* Ion channels
* Transporters
* Xenobiotic metabolising enzymes

In [30]:
def f(key, group):
    
    symbols = group['symbol'].values
        
    return {'target': key, 'symbols': symbols, 'count': len(symbols)}
    
pd.DataFrame([f(x, y) for x, y in sorted(symbols.loc[lost].groupby('targets'), key=lambda x: float(x[0][0][1]))], columns=['target', 'symbols', 'count'])

,target,symbols,count
0,"((Potassium voltage-gated channel KQT-like member 1 and minimal potassium channel MinK, 1),)",[KCNE1],1
1,"((ANF, 2.2),)","[NPR1, NPR2]",2
2,"((CGRP1, 2.2),)",[CALCR],1
3,"((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))","[ATP1A4, ATP1A2, ATP1B4, ATP1B2, ATP1B3, ATP1B1, ATP1A3, ATP1A1]",8
4,"((tyrosine hydroxylase, 3),)",[TH],1
5,"((IK1, 4),)","[KCNJ12, KCNJ2]",2
6,"((IKP, 4),)","[KCNK1, KCNK4, KCNK6, KCNK3]",4
7,"((IKur, 4),)",[KCNC1],1
8,"((If, 4),)","[HCN2, HCN4]",2
9,"((Ito,f, 4),)","[KCND3, KCND2]",2


In [31]:
symbols.query("symbol in @lost")

,symbol,approved_name,targets
symbol,,,
ABCB11,ABCB11,"ATP-binding cassette, sub-family B (MDR/TAP), member 11","((BSEP, 6), (BSEP, 7), (BSEP, 8))"
ABCB4,ABCB4,"ATP-binding cassette, sub-family B (MDR/TAP), member 4","((MDR3, 7), (MDR3, 8))"
ABCC2,ABCC2,"ATP-binding cassette, sub-family C (CFTR/MRP), member 2","((MRP2, 7), (MRP2, 8), (ATP-binding cassette, sub-family C (CFTR/MRP), member 2, 11))"
ABCC3,ABCC3,"ATP-binding cassette, sub-family C (CFTR/MRP), member 3","((MRP3, 7), (MRP3, 8))"
ABCC4,ABCC4,"ATP-binding cassette, sub-family C (CFTR/MRP), member 4","((MRP4, 7), (MRP4, 8))"
ABCC6,ABCC6,"ATP-binding cassette, sub-family C (CFTR/MRP), member 6","((MRP6, 8),)"
ABCG5,ABCG5,"ATP-binding cassette, sub-family G (WHITE), member 5","((ABCG5, 7),)"
ABCG8,ABCG8,"ATP-binding cassette, sub-family G (WHITE), member 8","((ABCG8, 7),)"
ATP1A1,ATP1A1,"ATPase, Na+/K+ transporting, alpha 1 polypeptide","((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))"


In [32]:
def f(x):
        
    targets = x['targets']
                 
    return pd.DataFrame({'symbol': [x['symbol']]*len(targets), 'table': [y[1] for y in targets]})

subset = symbols.query("symbol in @lost")

tables = pd.concat(f(x) for i, x in subset.iterrows())

In [33]:
tables['table'].value_counts()

8      16
11     16
7      15
4      14
3       9
10      8
9       4
6       4
2.2     3
5       2
1       1
dtype: int64

### Save/restore

File now includes per-target counts.

In [34]:
backup_file('chembl_targets.pkl')

targets.to_pickle('chembl_targets.pkl')

In [35]:
# targets = pd.read_pickle('chembl_targets.pkl')

# targets.shape